In [1]:
from bokeh.plotting import figure, save, show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColumnDataSource, Range1d
from bokeh.resources import CDN
from bokeh.tile_providers import CARTODBPOSITRON_RETINA
import pysal as ps
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

c:\users\user\appdata\local\programs\python\python36\lib\site-packages\pysal\__init__.py:65: VisibleDeprecationWarning: PySAL's API will be changed on 2018-12-31. The last release made with this API is version 1.14.4. A preview of the next API version is provided in the `pysal` 2.0 prelease candidate. The API changes and a guide on how to change imports is provided at https://pysal.org/about
  ), VisibleDeprecationWarning)


In [2]:
PATH = r"mapdata/TOWN_MOI_1071031.shp"
villages_shp = gpd.read_file(PATH, encoding='utf-8') #全台灣村里界圖
villages_shp = villages_shp.to_crs({'init' :'epsg:4326'})

In [3]:
villages_shp.head()

,TOWNID,TOWNCODE,COUNTYNAME,TOWNNAME,TOWNENG,COUNTYID,COUNTYCODE,geometry
0,V02,10014020,臺東縣,成功鎮,Chenggong Township,V,10014,"POLYGON ((121.409815737 23.21369278500009, 121..."
1,T21,10013210,屏東縣,佳冬鄉,Jiadong Township,T,10013,"POLYGON ((120.5484580210001 22.46067220300006,..."
2,P13,10009130,雲林縣,麥寮鄉,Mailiao Township,P,10009,"POLYGON ((120.3019750420001 23.81624614400005,..."
3,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"(POLYGON ((121.491547243 22.67745703900005, 12..."
4,V16,10014160,臺東縣,蘭嶼鄉,Lanyu Township,V,10014,(POLYGON ((121.6117956420001 21.94289657000007...


In [4]:
def cartesian(x): 
    return np.vstack(np.array([np.array(np.meshgrid(*i)).T.reshape(-1,8) for i in x.values]))

In [ ]:
villages_shp = gpd.GeoDataFrame(cartesian(villages_shp),columns=villages_shp.columns)

In [ ]:
villages_shp.head(6)

,TOWNID,TOWNCODE,COUNTYNAME,TOWNNAME,TOWNENG,COUNTYID,COUNTYCODE,geometry
0,V02,10014020,臺東縣,成功鎮,Chenggong Township,V,10014,"POLYGON ((121.409815737 23.21369278500009, 121..."
1,T21,10013210,屏東縣,佳冬鄉,Jiadong Township,T,10013,"POLYGON ((120.5484580210001 22.46067220300006,..."
2,P13,10009130,雲林縣,麥寮鄉,Mailiao Township,P,10009,"POLYGON ((120.3019750420001 23.81624614400005,..."
3,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.491547243 22.67745703900005, 121..."
4,V11,10014110,臺東縣,綠島鄉,Lvdao Township,V,10014,"POLYGON ((121.512131544 22.68070962500008, 121..."
5,V16,10014160,臺東縣,蘭嶼鄉,Lanyu Township,V,10014,"POLYGON ((121.6117956420001 21.94289657000007,..."


In [ ]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [ ]:
villages_shp['x'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
villages_shp['y'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [ ]:
#villages_shp[['x', 'y']].head(6)
villages_shp.head(6)

In [ ]:
#villages_shp_source = GeoJSONDataSource(geojson=villages_shp.to_json())
source = ColumnDataSource(data=dict(
    lat=villages_shp.x,
    lon=villages_shp.y,
    cty=villages_shp.COUNTYNAME + ", " + villages_shp.TOWNNAME
))

TOOLTIPS = [
    ("Latitude", "$lat"),
    ("Longitude", "$lon"),
    ("County", "@cty"),   
]
#HoverTool.mode="vline"

In [ ]:
# Initialize our plot figure
p = figure(title="Taiwan Map", output_backend="webgl", plot_width=800, plot_height=600,
          tools="pan,wheel_zoom,box_zoom,reset,hover",
          tooltips=TOOLTIPS)

In [ ]:
# Add the lines to the map from our GeoJSONDataSource -object
p.multi_line('x', 'y', source=source, color='black', line_width=2)

In [ ]:
#output_file("taiwanmap.html")
output_notebook()

In [ ]:
show(p)
#save(p, title="taiwanmap")